# COMP90051 Assignment 2 Data Exploration

In [1]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

In [2]:
#setting working directory
import os
os.chdir('/Users/fungs4/Desktop/Skye/University of Melbourne/Year 1 2025/COMP90051 Statistical Machine Learning/COMP90051 Assignments/COMP90051 Assignment 2/Shoe vs Sandal vs Boot Dataset')
print(os.getcwd())

/Users/fungs4/Desktop/Skye/University of Melbourne/Year 1 2025/COMP90051 Statistical Machine Learning/COMP90051 Assignments/COMP90051 Assignment 2/Shoe vs Sandal vs Boot Dataset


In [3]:
#loading image paths
data_dir = Path('.')

boot_images = list((data_dir / 'Boot').glob('*.jpg')) + list((data_dir / 'Boot').glob('*.png'))
sandal_images = list((data_dir / 'Sandal').glob('*.jpg')) + list((data_dir / 'Sandal').glob('*.png'))
shoe_images = list((data_dir / 'Shoe').glob('*.jpg')) + list((data_dir / 'Shoe').glob('*.png'))

print(f"Boot: {len(boot_images)} images")
print(f"Sandal: {len(sandal_images)} images")
print(f"Shoe: {len(shoe_images)} images")
print(f"Total: {len(boot_images) + len(sandal_images) + len(shoe_images)} images")

Boot: 5000 images
Sandal: 5000 images
Shoe: 5000 images
Total: 15000 images


In [4]:
#combining image paths, creating labels
image_paths = boot_images + sandal_images + shoe_images
all_labels = ['Boot'] * len(boot_images) + ['Sandal'] * len(sandal_images) + ['Shoe'] * len(shoe_images)

Creating train/test split

In [5]:
from sklearn.model_selection import train_test_split

train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, 
    all_labels, 
    test_size = 0.2, 
    stratify = all_labels,
    random_state = 66
)

print(f"Training set: {len(train_paths)} images")
print(f"Test set: {len(test_paths)} images")

#checking class distribution (making sure train/test sets have the same proportion of each class)
from collections import Counter
print("\nTraining set distribution:", Counter(train_labels))
print("Test set distribution:", Counter(test_labels))

Training set: 12000 images
Test set: 3000 images

Training set distribution: Counter({'Boot': 4000, 'Sandal': 4000, 'Shoe': 4000})
Test set distribution: Counter({'Shoe': 1000, 'Sandal': 1000, 'Boot': 1000})


# Research Question

**"When trained on augmented data, which machine learning architectures are most robust to progressively severe real-world distortions in footwear classification?"**

We want to look at which architectures handle distortions/noise best, when given augmented training data. The approach is as follows
1. Apply moderate augmentation/distortions to the training set
2. Create 5 severity levels for each distortion type, for the test set
    * For example, our first test set will be clean. Second test set will have mild distortions,...,last test set will have extreme distortions
3. Train 3 models on augmented data
    * This phase will include CV/Hyperparameter tuning
4. Testing
    * Test each model on each of the 5 test sets, with the goal of exploring how model performance deviates, as the images that we feed in have increasing levels of distortion

An alternative approach, adds in the step of training another set of 3 models, on clean data only. This allows us to answer/investigate the following:
* Real world images are noisy. We want our models to be robust, even when trained on/tested on images that have natural distortions/occlusions. The question we want to answer is "is augmentation necessary for robustness". In other words, is it possible to train good robust models for image classification, when they are trained only on "clean" images? We would essentially be investigating this by training models on clean images, and seeing how they perform when they are given noisy/distorted images to classify. If these models still perform well, than we can say that certain model architectures are able to learn relevant features from clean images, and are not distracted by the added noise from the test images, when actually classifying unseen images.
Based on this, an alternative research question would be:

**"How does architectural choice affect robustness to real-world distortions: Is robustness inherent to the architecture or learned from augmented training?"**

In summary: 
* Condition A - Train on clean data -> Tests inherent robustness, which architectures naturally generalise
* Condition B - Train on augmented data -> Tests learned robustness, which architectures learn distortion-invariance explicitly, and are able to benefit most from being trained on augmented images, that are more similar (in that they have noise/occlusions/distortions) to real-world images that we wish to classify

## What distortions/occlusions/noise to add? 

A realistic approach to adding distortions is as follows: The images that are currently in our dataset are very clean. More specifically, they depict footwear on a white background, with almost no noise. However, this is not a very realistic scenario. There may be many cases where we would want to classify footwear, where the images contains a lot of noise. Examples would be natural images of people wearing shoes. In these cases since we are only interested in classifying the footwear type, the other parts of the image (e.g. the background, what the individual is wearing, etc.) are all effectively noise. Hence, it makes sense to add noise to the images. 